# Adaptive robust lot sizing

In [1]:
import cvxpy as cp
import numpy as np
import torch
import matplotlib.pyplot as plt

import lropt
from lropt.parameter import Parameter
from lropt.robust_problem import RobustProblem
from lropt.uncertain import UncertainParameter

np.random.seed(seed=1234)

In [3]:
N = 30
c = 20
dmax = 20
Gamma = 20*np.sqrt(N)
coordinates = 10*np.random.rand(2, N)
t = ((coordinates[[0]] - coordinates[[0]].T) ** 2
     + (coordinates[[1]] - coordinates[[1]].T) ** 2) ** 0.5

In [6]:
uncertainty_set = lropt.Ellipsoidal(rho=1, p=Gamma, lb=0, ub=dmax)
x = cp.Variable(N)
y = cp.Variable((N,N))
d = UncertainParameter(N, uncertainty_set=uncertainty_set)

objective = cp.Minimize(cp.sum(c*x) + cp.sum(t@y))
constraints = [
                d <= cp.sum(y, axis=0) - cp.sum(y, axis=1) + x,
                y >= 0,
                x >= 0,
                x <= c,
              ]